# 영화 필터링

- 이상한 영화들이 존재하여 제거해야함.

In [ ]:
import pandas as pd

movie_metadata = pd.read_excel("cinemate_data_0511.xlsx")
final_metadata = pd.read_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")

In [ ]:
final_metadata

In [ ]:
# movie_metadata 내에서 title이 영어이거나 한국어가 아닌 것들
movie_metadata[~(movie_metadata['title'].str.contains('[a-zA-Z]') | movie_metadata['title'].str.contains('[가-힣]'))]

In [ ]:
# movie_metadata 내에서 title이 영어이거나 한국어인 것들의 인덱스로 final_metadata에서 해당 데이터만 추출
final_metadata = final_metadata.iloc[movie_metadata[(movie_metadata['title'].str.contains('[a-zA-Z]') | movie_metadata['title'].str.contains('[가-힣]'))].index]

In [ ]:
final_metadata.to_excel("final_metadata_with_overview_embeddings_only_eng_kor_0528.xlsx", index=False)

## 변경한 데이터의 장르 정보 확인

In [ ]:
# 각 영화의 인덱스를 키로, 해당 영화의 장르를 값으로 가지는 딕셔너리 생성
movie_genres = {}
for index, row in final_metadata.iterrows():
    genres = row.iloc[5].split(',')  # ','로 장르 분리
    movie_genres[index] = set(genres)  # 장르 집합 생성하여 딕셔너리에 저장
# movie_genres
# 각 장르별 영화 개수 plot
import matplotlib.pyplot as plt

genre_count = {}
for genres in movie_genres.values():
    for genre in genres:
        if genre in genre_count:
            genre_count[genre] += 1
        else:
            genre_count[genre] = 1
genre_count = dict(sorted(genre_count.items(), key=lambda x: x[1], reverse=True))
plt.figure(figsize=(20, 10))
plt.bar(genre_count.keys(), genre_count.values())
plt.xticks(rotation=45)
plt.show()

In [ ]:
genre_count

In [ ]:
# final_metadata 초기화
final_metadata.reset_index(drop=True, inplace=True)

In [ ]:
final_metadata

# 장르 one-hot encoding

In [ ]:
genres_one_hot_encoded = final_metadata.genres.str.get_dummies(sep=',')
genres_one_hot_encoded

# 감독, 배우 one-hot encoding

In [ ]:
unique_directors_combinations = final_metadata['directors'].unique()
# unique_directors_combinations.__len__()
unique_directors = set()

for combination in unique_directors_combinations:
    directors = combination.split(',')
    unique_directors.update(directors)

print(sorted(unique_directors))
directors_one_hot_encoded = final_metadata.directors.str.get_dummies(sep=',')
# final_metadata[directors_one_hot_encoded.sum(axis=1) > 1]
directors_one_hot_encoded

In [ ]:
unique_actors_combinations = final_metadata['actors'].unique()
unique_actors = set()

for combination in unique_actors_combinations:
    actors = combination.split(',')
    unique_actors.update(actors)

print(sorted(unique_actors))
actors_one_hot_encoded = final_metadata.actors.str.get_dummies(sep=',')
# final_metadata[actors_one_hot_encoded.sum(axis=1) > 1]
actors_one_hot_encoded

# PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
directors_pca = pca.fit_transform(directors_one_hot_encoded)
directors_pca_df = pd.DataFrame(directors_pca, columns=[f'director_pca_{i+1}' for i in range(10)])

actors_pca = pca.fit_transform(actors_one_hot_encoded)
actors_pca_df = pd.DataFrame(actors_pca, columns=[f'actor_pca_{i+1}' for i in range(10)])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
genres_pca = pca.fit_transform(genres_one_hot_encoded)
genres_pca_df = pd.DataFrame(genres_pca, columns=[f'genre_pca_{i+1}' for i in range(10)])
genres_pca_df

In [ ]:
final_features = pd.concat([directors_pca_df, actors_pca_df, genres_pca_df * 2, final_metadata.iloc[:, -384:]], axis=1)
final_features

In [ ]:
import numpy as np

np.save('final_features_0528.npy', final_features.to_numpy())

In [ ]:
movie_metadata = movie_metadata[(movie_metadata['title'].str.contains('[a-zA-Z]') | movie_metadata['title'].str.contains('[가-힣]'))]

In [ ]:
movie_metadata.reset_index(drop=True, inplace=True)
movie_metadata

In [ ]:
movie_metadata.to_excel("cinemate_data_only_eng_kor_0528.xlsx", index=False)

## nan 값 처리

In [ ]:
movie_metadata.loc[movie_metadata.backdrop_path.isnull(), 'backdrop_path'] = None
movie_metadata.loc[movie_metadata.poster_path.isnull(), 'poster_path'] = None
movie_metadata.loc[movie_metadata.overview.isnull(), 'overview'] = ""

# SQL 문 생성

In [ ]:
genre_mapping = {
    "Action": 1,
    "Adventure": 2,
    "Animation": 3,
    "Biography": 4,
    "Comedy": 5,
    "Crime": 6,
    "Documentary": 7,
    "Drama": 8,
    "Family": 9,
    "Fantasy": 10,
    "History": 11,
    "Horror": 12,
    "Music": 13,
    "Musical": 14,
    "Mystery": 15,
    "Romance": 16,
    "Sci-Fi": 17,
    "Sport": 18,
    "Thriller": 19,
    "War": 20,
    "Western": 21
}

In [ ]:
# 영화 데이터 삽입
movie_insert_sql = """
INSERT INTO movie (id, movie_id, rating, backdrop_path, original_title, movie_title, release_date, poster_path, overview)
VALUES ({}, {}, {}, '{}', '{}', '{}', '{}', '{}', '{}');
"""

movies_sql_statements = []

for index, row in movie_metadata.iterrows():
    print(row['backdrop_path'])
    backdrop_path = "https://image.tmdb.org/t/p/original" + row['backdrop_path'] if row['backdrop_path'] else None
    poster_path = "https://image.tmdb.org/t/p/original" + row['poster_path'] if row['poster_path'] else None
    movie_data = (
        index + 1,
        row['movie_id'],
        row['rating'],
        backdrop_path,
        row['original_title'].replace("'", "''"),
        row['title'].replace("'", "''"),
        row['release_date'],
        poster_path,
        row['overview'].replace("'", "''")
    )
    movie_sql = movie_insert_sql.format(*movie_data)
    movies_sql_statements.append(movie_sql)

# # 장르 데이터 삽입
# genre_insert_sql = """
# INSERT INTO genre (genre_name)
# VALUES ('{}');
# """

# genres_sql_statements = []

# # 장르 데이터를 중복 없이 추출
# genres_set = set()
# for index, row in cinemate_data.iterrows():
#     genres = row['genres'].split(',')
#     for genre in genres:
#         genres_set.add(genre.strip())

# # 중복 없는 장르 데이터를 DB에 삽입
# for genre in genres_set:
#     genre_sql = genre_insert_sql.format(genre.strip())
#     genres_sql_statements.append(genre_sql)

# 영화와 장르의 관계 데이터 삽입
movie_genre_mapping_insert_sql = """
INSERT INTO genre_movie (genre_id, movie_id)
VALUES ({}, {});
"""

movie_genre_sql_statements = []

for index, row in final_metadata.iterrows():
    genres = row['genres'].split(',')
    for genre in genres:
        genre_id = genre_mapping[genre]
        movie_genre_sql = movie_genre_mapping_insert_sql.format(genre_id, index + 1)
        movie_genre_sql_statements.append(movie_genre_sql)

# SQL 쿼리를 파일에 쓰기
with open('data_ver2.sql', 'w') as f:
    for sql_statement in movies_sql_statements + movie_genre_sql_statements:
        f.write(sql_statement + '\n')